# Documentation
**Spencer Ressel**

*2023.01.17*

***

This script is a compilation of functions that serve to recreate the figures of 
two papers, Wheeler and Hendon (2004) and Jiang et al. (2020).
The Wheeler and Hendon (2004) paper describes the creation of a real-time multivariate
MJO index (RMM), and splits the MJO into 8 phases based on the RMM index. 
The Jiang et al. (2020) paper is a broad overview of the MJO,
where the specific figure (Fig. 1) recreated is a composite of MJO-associated rainfall
anomalies during boreal winter for each MJO phase.

***

### Inputs:       
#### Global 2.5° x 2.5° resolution, daily timeseries in netCDF format:
* precipitation from TRMM
* outgoing longwave radiation (OLR) data from Liebmann and Smith (1996)
* 200 hPa zonal and meridional wind from ERA5 reanalysis
* 850 hPa zonal and meridional wind from ERA5 reanalysis
                
### Dependencies: 
* mjo_mean_state_diagnostics.py

# Imports

In [2]:
# Data processing tools
import numpy as np
import scipy
import scipy.signal as signal
import xarray as xr
import mjo_mean_state_diagnostics as mjo

# Plotting
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.gridspec import GridSpec

# Seaborn
import seaborn as sns

# # Cartopy
# import cartopy.crs as ccrs
# import cartopy.util as cutil
# from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

# Set Physical Constants and Analysis Parameters

In [165]:
# Set time bounds
TIME_MIN = '1998-01-01T00:00:00.000000000'
TIME_MAX = '2018-12-31T00:00:00.000000000'
SAMPLING_FREQUENCY = 1

# Set latitude bounds
LATITUDE_SOUTH = -25
LATITUDE_NORTH = 25

# Set central longitude
CENTRAL_LONGITUDE = 160

# Set longitude bounds
LONGITUDE_MIN = -180
LONGITUDE_MAX = 180

# Cut-off periods for intraseasonal filtering
INTRASEASONAL_LOWCUT = 100
INTRASEASONAL_HIGHCUT = 20

# Seconds per day
SECONDS_PER_DAY = 24 * 3600

# Load Data

In [198]:
# TRMM Precipitation
data_directory_precip = r"/home/disk/eos7/sressel/Research/Data/NASA/TRMM/"
file_name_precip = r"trmm_precipitation_daily_1998_2018.nc"
data_precipitation = xr.open_dataset(
    data_directory_precip + file_name_precip, engine="netcdf4"
)
precipitation = data_precipitation['prec']

# NASA OLR (Liebmann and Smith 1996)
data_directory_olr = r"/home/disk/eos7/sressel/Research/Data/NOAA/"
file_name_olr = r"olr.day.mean.nc"
data_olr = xr.open_dataset(data_directory_olr + file_name_olr, engine="netcdf4")
outgoing_longwave_radiation = data_olr['olr']

# Wind 
data_directory_wind = r"/home/disk/eos7/sressel/Research/Data/ECMWF/ERA5/"
file_name_wind = r"era5_combined_wind_daily_1979_2020_pressure_levels.nc"
wind_data = xr.open_dataset(data_directory_wind + file_name_wind, engine='netcdf4')

# Upper and lower level zonal wind
upper_level_zonal_wind = wind_data['u'].sel(lev=200)
lower_level_zonal_wind = wind_data['u'].sel(lev=850)

# Upper and lower level zonal wind
upper_level_meridional_wind = wind_data['v'].sel(lev=200)
lower_level_meridional_wind = wind_data['v'].sel(lev=850)

time = outgoing_longwave_radiation.time
latitude = outgoing_longwave_radiation.lat
longitude = outgoing_longwave_radiation.lon

variables_dict = {
    # 'precipitation' : precipitation,
    'outgoing longwave radiation' : outgoing_longwave_radiation,
    'upper level zonal wind' : upper_level_zonal_wind,
    'lower level zonal wind' : lower_level_zonal_wind,
    'upper level meridional wind' : upper_level_meridional_wind,
    'lower level meridional wind' : lower_level_meridional_wind,
}

# Process Data

Detrend the data, remove the annual cycle and the first three harmonics (seasonal cycle), and filter the data on intraseasonal timescales

## Subset Data
Specifically select the data from times of interest and from tropical latitudes

In [199]:
variables_subset = {}

for variable in variables_dict:
    variables_subset[variable] = variables_dict[variable].sel(
        time=slice(TIME_MIN, TIME_MAX),
        lat=slice(LATITUDE_NORTH, LATITUDE_SOUTH)
    )

## Detrend the data

In [200]:
variables_detrended = {}

for variable in variables_dict:
    variables_detrended[variable] = variables_subset[variable].copy(deep=True)
    variables_detrended[variable].values = signal.detrend(
                                               variables_subset[variable], 
                                               axis=0, 
                                               type='linear'
                                           )

## Remove the Annual Cycle

In [202]:
variables_deannualized = {}
variables_annual_cycle = {}

for variable in variables_dict:
    variables_deannualized[variable] = variables_detrended[variable].copy(deep=True)
    
    [           
        variables_deannualized[variable].values,
        variables_annual_cycle[variable].values,
    ] = mjo.remove_annual_cycle_matrix(
        variables_detrended[variable], 
        time = variables_detrended[variable].time.values,
        lat =  variables_detrended[variable].lat.values,
        lon =  variables_detrended[variable].lon.values
    )    

KeyError: 'outgoing longwave radiation'

## Filter Data 

Temporally filter the data on intraseasonal (20-100 day) timescales, using a Lanczos filter